In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import cv2  
from tqdm import tqdm
from random import randint
import json
import matplotlib.image as mpimg
from matplotlib import rcParams
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.preprocessing.image import load_img 
from keras.preprocessing.image import img_to_array 
from tensorflow.keras.applications.vgg16 import preprocess_input,VGG16

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

Using TensorFlow backend.


In [2]:
def show_image(images,name,row,col,size1=15,size2=15):
    n = len(images)
    plt.subplots(figsize=(15, 15))

    for i in range(n):
        plt.subplot(row,col,i+1)
        plt.title(name[i])
        plt.imshow(images[i], cmap='gray')
        plt.axis('off')   
    plt.show()

In [2]:
train_path = "data/train"
test_path = "data/test"

In [3]:
train = os.listdir(train_path)
test = os.listdir(test_path)

In [4]:
train_image = [cv2.resize(cv2.cvtColor(cv2.imread(os.path.join(train_path, image)), cv2.COLOR_BGR2RGB),(150,150))  for image in train ]

In [5]:
test_image = [cv2.resize(cv2.cvtColor(cv2.imread(os.path.join(test_path, image)), cv2.COLOR_BGR2RGB),(150,150))  for image in test ]

In [6]:
label = pd.read_csv("data/train_features.csv")
label[["x","y"]]

,x,y
0,-9.380678,3.58272
1,-9.380678,3.58272
2,-9.380678,3.58272
3,-9.380678,3.58272
4,-9.380678,3.58272
...,...,...
7495,-4.680678,35.18272
7496,-4.680678,35.18272
7497,-4.680678,35.18272
7498,-4.680678,35.18272


In [7]:
y_label = []
for index, row in label.iterrows():
    y_label.append((row["x"],row["y"]))
y_label = np.array(y_label)
train_image = np.array(train_image)

In [8]:
test_image = np.array(test_image)

In [9]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [10]:
train_image.shape

(7500, 150, 150, 3)

In [11]:
y_label.shape

(7500, 2)

In [12]:
vgg16 = VGG16(weights='imagenet',input_shape=[150,150,3],include_top=False)
vgg19 = VGG19(weights='imagenet',input_shape=[150,150,3],include_top=False)
resnet50 = ResNet50(weights='imagenet',input_shape=[150,150,3],include_top=False)

In [13]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers.merge import concatenate
from tensorflow.keras.models import Model

In [14]:
def train_model(pre_train):
  for layer in pre_train.layers:
    layer.trainable = False
  layer = layers.Conv2D(filters = 32,kernel_size = (1,1),padding = "same",activation = "relu")(pre_train.layers[-1].output)
  # layer = layers.Conv2D(filters = 64,kernel_size = (3,3),padding = "same",activation = "relu")(layer)
  # layer = layers.Conv2D(filters = 64,kernel_size = (3,3),padding = "same",activation = "relu")(layer)
  layer = layers.MaxPooling2D(pool_size=(2,2), strides=2,padding="same")(layer)
  layer = layers.Flatten()(layer)
  layer = layers.Dropout(0.5)(layer)
  #layer = layers.Dense(1024,activation="relu")(layer)
  layer = layers.Dense(2,activation="linear")(layer)

  model = tf.keras.Model(inputs = pre_train.inputs,outputs = layer)

  return model


In [15]:
model_vgg16 = train_model(vgg16)

opt = keras.optimizers.Adam(learning_rate=0.001)
model_vgg16.compile(optimizer=opt,
            loss=tf.keras.losses.MeanAbsoluteError())
            
model_vgg16.fit(train_image, label[["x","y"]], epochs=10, batch_size=128)

Epoch 1/10
59/59 [==============================] - 38s 414ms/step - loss: 40.2066
Epoch 2/10
59/59 [==============================] - 18s 298ms/step - loss: 33.9632
Epoch 3/10
59/59 [==============================] - 18s 300ms/step - loss: 31.7543
Epoch 4/10
59/59 [==============================] - 18s 300ms/step - loss: 30.2987
Epoch 5/10
59/59 [==============================] - 18s 301ms/step - loss: 29.0690
Epoch 6/10
59/59 [==============================] - 18s 302ms/step - loss: 28.3851
Epoch 7/10
59/59 [==============================] - 18s 302ms/step - loss: 27.7495
Epoch 8/10
59/59 [==============================] - 18s 305ms/step - loss: 27.0521
Epoch 9/10
59/59 [==============================] - 18s 307ms/step - loss: 26.6590
Epoch 10/10
59/59 [==============================] - 18s 303ms/step - loss: 26.1442


In [17]:
model_vgg19 = train_model(vgg19)

opt = keras.optimizers.Adam(learning_rate=0.001)
model_vgg19.compile(optimizer=opt,
            loss=tf.keras.losses.MeanAbsoluteError())

model_vgg19.fit(train_image, label[["x","y"]], epochs=10, batch_size=128)

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:GPU:0 to /job:localhost/replica:0/task:0/device:CPU:0 in order to run TensorDataset: Dst tensor is not initialized. [Op:TensorDataset]

In [ ]:
model_resnet = train_model(resnet50)

opt = keras.optimizers.Adam(learning_rate=0.001)
model_resnet.compile(optimizer=opt,
            loss=tf.keras.losses.MeanAbsoluteError())

model_resnet.fit(train_image, label[["x","y"]], epochs=10, batch_size=128)

In [ ]:
result = model.predict(test_image)


In [ ]:
x = []
y = []
for i in result:
   x.append(i[0])
   y.append(i[1]) 

In [ ]:
test_result = pd.read_csv("data/imagenames.csv")

In [ ]:
test_result["x"] = x
test_result["y"] = y

In [ ]:
test_result.to_csv("result/CNN_baseline_40.csv",index=False)